# Flu Shot Learning: Predict H1N1 and Seasonal Flu Vaccines

https://www.drivendata.org/competitions/66/flu-shot-learning/

In [255]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import xgboost

from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef

from skrebate import ReliefF

# Load data

In [7]:
frames = dict()
for i in Path('../data').glob('**/*.csv'):
    frames[i.stem] = pd.read_csv(i)
    print(i.stem)

submission_format
test_set_features
training_set_features
training_set_labels


In [185]:
X_test = frames['test_set_features']
X = frames['training_set_features'].drop("respondent_id", axis=1)
y = frames['training_set_labels']

# Contents

In [186]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_concern                 26615 non-null  float64
 1   h1n1_knowledge               26591 non-null  float64
 2   behavioral_antiviral_meds    26636 non-null  float64
 3   behavioral_avoidance         26499 non-null  float64
 4   behavioral_face_mask         26688 non-null  float64
 5   behavioral_wash_hands        26665 non-null  float64
 6   behavioral_large_gatherings  26620 non-null  float64
 7   behavioral_outside_home      26625 non-null  float64
 8   behavioral_touch_face        26579 non-null  float64
 9   doctor_recc_h1n1             24547 non-null  float64
 10  doctor_recc_seasonal         24547 non-null  float64
 11  chronic_med_condition        25736 non-null  float64
 12  child_under_6_months         25887 non-null  float64
 13  health_worker   

In [188]:
X.head(10)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb
5,3.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,atmpeygn,"MSA, Principle City",2.0,3.0,saaquncn,vlluhbov
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",0.0,0.0,xicduogh,xtkaffoo
7,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,bhuqouqj,Non-MSA,2.0,0.0,pxcmvdjn,xqwwgdyp
8,0.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,"> $75,000",Married,Own,Employed,bhuqouqj,"MSA, Not Principle City",1.0,0.0,xicduogh,ccgxvspp
9,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,"MSA, Not Principle City",0.0,0.0,NaN,NaN


#### Describe categoricals

In [189]:
X.select_dtypes('object').describe(include='all')

,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,employment_industry,employment_occupation
count,26707,25300,26707,26707,22284,25299,24665,25244,26707,26707,13377,13237
unique,5,4,4,2,3,2,2,3,10,3,21,23
top,65+ Years,College Graduate,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,lzgpxyit,"MSA, Not Principle City",fcxhlnwr,xtkaffoo
freq,6843,10097,21222,15858,12777,13555,18736,13560,4297,11645,2468,1778


#### Describe numericals

In [190]:
X.select_dtypes('number').describe()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_adults,household_children
count,26615.000000,26591.000000,26636.000000,26499.000000,26688.000000,26665.000000,26620.00000,26625.000000,26579.000000,24547.000000,...,25903.000000,14433.00000,26316.000000,26319.000000,26312.000000,26245.000000,26193.000000,26170.000000,26458.000000,26458.000000
mean,1.618486,1.262532,0.048844,0.725612,0.068982,0.825614,0.35864,0.337315,0.677264,0.220312,...,0.111918,0.87972,3.850623,2.342566,2.357670,4.025986,2.719162,2.118112,0.886499,0.534583
std,0.910311,0.618149,0.215545,0.446214,0.253429,0.379448,0.47961,0.472802,0.467531,0.414466,...,0.315271,0.32530,1.007436,1.285539,1.362766,1.086565,1.385055,1.332950,0.753422,0.928173
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,1.00000,3.000000,1.000000,1.000000,4.000000,2.000000,1.000000,0.000000,0.000000
50%,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,1.000000,0.000000,...,0.000000,1.00000,4.000000,2.000000,2.000000,4.000000,2.000000,2.000000,1.000000,0.000000
75%,2.000000,2.000000,0.000000,1.000000,0.000000,1.000000,1.00000,1.000000,1.000000,0.000000,...,0.000000,1.00000,5.000000,4.000000,4.000000,5.000000,4.000000,4.000000,1.000000,1.000000
max,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,...,1.000000,1.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,3.000000


# Missing values

In [191]:
(100 * X.isna().sum() / X.shape[0]).sort_values(ascending=False)

employment_occupation          50.436215
employment_industry            49.912008
health_insurance               45.957989
income_poverty                 16.561201
doctor_recc_h1n1                8.087767
doctor_recc_seasonal            8.087767
rent_or_own                     7.645936
employment_status               5.477965
marital_status                  5.272026
education                       5.268282
chronic_med_condition           3.635751
child_under_6_months            3.070356
health_worker                   3.010447
opinion_seas_sick_from_vacc     2.010709
opinion_seas_risk               1.924589
opinion_seas_vacc_effective     1.729884
opinion_h1n1_sick_from_vacc     1.479013
opinion_h1n1_vacc_effective     1.464036
opinion_h1n1_risk               1.452803
household_children              0.932340
household_adults                0.932340
behavioral_avoidance            0.778822
behavioral_touch_face           0.479275
h1n1_knowledge                  0.434343
h1n1_concern    

# Feature selection

1. Variance 
1. Random forrest feature importance
1. Mutual information
1. Matthews correlation (binary features only)
1. Relieff

### Variance

In [192]:
# Reference std
e = np.ones(X.dropna().shape[0]*3)*0.5
std_ref = np.sum(e**2) / len(e)

for i in sorted(set(X.dropna().nunique().values), reverse=True):
    x = np.random.randint(i, size=(1000000))
    std = np.sqrt(np.sum((x-x.mean())**2) / len(x))
    print(f"{i}:\t {x.std():.6f}")

23:	 6.633176
21:	 6.055295
10:	 2.869887
5:	 1.414554
4:	 1.118513
3:	 0.816235
2:	 0.500000
1:	 0.000000


In [193]:
enc = OrdinalEncoder()
X_enc = enc.fit_transform(X.dropna())
X_enc = pd.DataFrame(X_enc, columns=X.columns, index=X.dropna().index)
df_std = pd.DataFrame([X_enc.std(), X_enc.nunique()], index=["std", "unique"]).transpose()
df_std.sort_values(['unique', 'std'], ascending=False)

,std,unique
employment_occupation,6.963490,23.0
employment_industry,6.679688,21.0
hhs_geo_region,2.840968,10.0
opinion_seas_risk,1.384711,5.0
opinion_h1n1_sick_from_vacc,1.321944,5.0
opinion_seas_sick_from_vacc,1.305995,5.0
opinion_h1n1_risk,1.285298,5.0
age_group,1.241749,5.0
opinion_seas_vacc_effective,1.086436,5.0
opinion_h1n1_vacc_effective,0.984741,5.0


### Random forest feature importance

In [301]:
clf = RandomForestClassifier(max_depth=5, n_jobs=-1)
X_temp = X.select_dtypes('number')
y_temp = y.loc[X_temp.dropna().index, :]
clf.fit(X_temp.dropna(), y_temp)

feature_importances = list(zip(X_temp.columns, clf.feature_importances_))
df_forest = pd.DataFrame(feature_importances, columns=['feature', 'importance'])
df_forest.set_index('feature', inplace=True)

#### Plot

In [361]:
fig = px.bar(df_forest.sort_values('importance'), x='importance')
fig.update_layout(title_text="Random Forest - feature importance")
fig.show()

### Mutual information

In [304]:
enc = OrdinalEncoder()
X_temp = X.dropna()
y_temp = y.loc[X.dropna().index]
X_temp = enc.fit_transform(X_temp)
X_temp = pd.DataFrame(X_temp, columns=X.columns, index=y_temp.index)
mi_h1n1 = mutual_info_classif(X_temp, y_temp['h1n1_vaccine'])
mi_seasonal = mutual_info_classif(X_temp, y_temp['seasonal_vaccine'])
df_mi = pd.DataFrame(list(zip(X_temp.columns, mi_h1n1, mi_seasonal)), columns=['feature', 'h1n1_vaccine', 'seasonal_vaccine'])
df_mi.set_index('feature', inplace=True)

#### Plot

In [306]:
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("h1n1", "seasonal"))

df_mi.sort_values('h1n1_vaccine', inplace=True)
fig.add_trace(
    go.Bar(y=df_mi['h1n1_vaccine'], x=df_mi.index, name='h1n1'),
    row=1, col=1
)

df_mi.sort_values('seasonal_vaccine', inplace=True)
fig.add_trace(
    go.Bar(y=df_mi['seasonal_vaccine'], x=df_mi.index, name='seasonal', marker_color='rgb(55, 83, 109)'),
    row=1, col=2
)

fig.update_layout(showlegend=False)
fig.show()

### Matthews correlation

Correlation coefficient for binary data.

In [309]:
unique_vals = {col: X.dropna()[col].unique().shape[0] for col in X.columns}
unique_vals = pd.Series(unique_vals)
binary_feats = unique_vals[unique_vals == 2].index

dropna_idx = X.dropna().index

enc = OrdinalEncoder()
X_enc = enc.fit_transform(X.loc[dropna_idx, binary_feats])
X_enc = pd.DataFrame(X_enc, columns=binary_feats, index=dropna_idx)

targets = ['h1n1_vaccine', 'seasonal_vaccine']
array = np.zeros((len(binary_feats), 2))
for itarget, target in enumerate(targets):
    for ifeature, feature in enumerate(binary_feats):
        coeff = matthews_corrcoef(X_enc.loc[dropna_idx, feature], y.loc[dropna_idx, target])
        array[ifeature, itarget] = coeff

df_matt = pd.DataFrame(array, columns=targets, index=binary_feats)
df_matt.rename_axis('feature', inplace=True)
df_matt.sort_values(targets[1], ascending=False)

,h1n1_vaccine,seasonal_vaccine
feature,,
doctor_recc_seasonal,0.231321,0.338884
doctor_recc_h1n1,0.374376,0.212517
health_worker,0.246153,0.208133
health_insurance,0.127450,0.168105
behavioral_wash_hands,0.114809,0.138967
chronic_med_condition,0.106065,0.130180
behavioral_touch_face,0.089193,0.118539
behavioral_avoidance,0.054768,0.069538
behavioral_face_mask,0.090719,0.048361


#### Plot

In [311]:
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("h1n1", "seasonal"))

df_matt.sort_values('h1n1_vaccine', inplace=True)
fig.add_trace(
    go.Bar(y=df_matt['h1n1_vaccine'], x=df_matt.index, name='h1n1'),
    row=1, col=1
)

df_matt.sort_values('seasonal_vaccine', inplace=True)
fig.add_trace(
    go.Bar(y=df_matt['seasonal_vaccine'], x=df_matt.index, name='seasonal', marker_color='rgb(55, 83, 109)'),
    row=1, col=2
)

fig.update_layout(showlegend=False)
fig.show()

### Relieff

In [299]:
fs = ReliefF(n_features_to_select=X.shape[1], n_neighbors=20, n_jobs=-1)
dropna_idx = X.dropna().index
X_enc = enc.fit_transform(X.loc[dropna_idx])

targets = ('h1n1_vaccine', 'seasonal_vaccine')
df_relieff = pd.DataFrame(X.columns, columns=['feature'])
for target in targets:
    fs.fit(X_enc, y.loc[dropna_idx, target].values)
    s = pd.Series(fs.feature_importances_, name=target)
    df_relieff = pd.concat([df_relieff, s], axis=1)

sort_idx = df_relieff[['h1n1_vaccine', 'seasonal_vaccine']].sum(axis=1).sort_values().index
df_relieff.sort_values('h1n1_vaccine', inplace=True)
df_relieff.set_index('feature', inplace=True)

,h1n1_vaccine,seasonal_vaccine
feature,,
employment_status,0.000000,0.000000
behavioral_face_mask,0.003511,0.000396
chronic_med_condition,0.005282,0.005981
behavioral_antiviral_meds,0.005678,0.002486
child_under_6_months,0.005795,0.000210
behavioral_large_gatherings,0.008995,0.003006
behavioral_outside_home,0.010090,0.002936
race,0.010929,0.009166
behavioral_wash_hands,0.013920,0.004412


#### Plot

In [300]:
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("h1n1", "seasonal"))

df_relieff.sort_values('h1n1_vaccine', inplace=True)
fig.add_trace(
    go.Bar(y=df_relieff['h1n1_vaccine'], x=df_relieff.index, name='h1n1'),
    row=1, col=1
)

df_relieff.sort_values('seasonal_vaccine', inplace=True)
fig.add_trace(
    go.Bar(y=df_relieff['seasonal_vaccine'], x=df_relieff.index, name='seasonal', marker_color='rgb(55, 83, 109)'),
    row=1, col=2
)

fig.update_layout(showlegend=False)
fig.show()

### Summarize correlations
1. Variance 
1. Random forrest feature importance
1. Mutual information
1. Matthews correlation (binary features only)
1. Relieff

In [322]:
corrs = ['forest', 'mutual_info', 'matthews_corr', 'relieff']
targets = ['h1n1_vaccine']*len(corrs) + ['seasonal_vaccine']*len(corrs)
tuples = list(zip(targets, corrs*2))
index = pd.MultiIndex.from_tuples(tuples, names=["vaccine", "method"])

targets = ['h1n1_vaccine', 'seasonal_vaccine']
big_df = pd.DataFrame()
for target in targets:
    big_df = pd.concat([big_df, df_forest, df_mi[target], df_matt[target], df_relieff[target]], axis=1)
big_df.columns=index
big_df

vaccine                     h1n1_vaccine                                      \
method                            forest mutual_info matthews_corr   relieff   
h1n1_concern                    0.012515    0.022629           NaN  0.036764   
h1n1_knowledge                  0.016617    0.022696           NaN  0.028119   
behavioral_antiviral_meds       0.000668    0.000000      0.020191  0.005678   
behavioral_avoidance            0.000737    0.010643      0.054768  0.019473   
behavioral_face_mask            0.000815    0.001556      0.090719  0.003511   
behavioral_wash_hands           0.001473    0.000440      0.114809  0.013920   
behavioral_large_gatherings     0.000499    0.000000      0.032941  0.008995   
behavioral_outside_home         0.000709    0.000000      0.032933  0.010090   
behavioral_touch_face           0.001849    0.007290      0.089193  0.018603   
doctor_recc_h1n1                0.127800    0.072189      0.374376  0.070988   
doctor_recc_seasonal            0.174299    0.020579      0.231321  0.014634   
chronic_med_condition           0.009102    0.009389      0.106065  0.005282   
child_under_6_months            0.000842    0.003725      0.073705  0.005795   
health_worker                   0.019751    0.033201      0.246153  0.032313   
health_insurance                0.034116    0.010356      0.127450  0.015636   
opinion_h1n1_vacc_effective     0.073411    0.070322           NaN  0.075159   
opinion_h1n1_risk               0.115046    0.106740           NaN  0.139327   
opinion_h1n1_sick_from_vacc     0.004146    0.000000           NaN  0.036756   
opinion_seas_vacc_effective     0.155682    0.037901           NaN  0.046015   
opinion_seas_risk               0.233205    0.080436           NaN  0.091611   
opinion_seas_sick_from_vacc     0.007409    0.005696           NaN  0.027412   
household_adults                0.002216    0.003942           NaN  0.020553   
household_children              0.007093    0.004572           NaN  0.017749   
education                            NaN    0.000000           NaN  0.029610   
hhs_geo_region                       NaN    0.000000           NaN  0.021167   
census_msa                           NaN    0.000000           NaN  0.020701   
employment_status                    NaN    0.004772           NaN  0.000000   
marital_status                       NaN    0.000000     -0.081329  0.025571   
race                                 NaN    0.006271           NaN  0.010929   
rent_or_own                          NaN    0.005269     -0.083579  0.015318   
sex                                  NaN    0.000000     -0.037421  0.017671   
income_poverty                       NaN    0.003251           NaN  0.024647   
age_group                            NaN    0.009596           NaN  0.037758   
employment_industry                  NaN    0.045694           NaN  0.021867   
employment_occupation                NaN    0.032538           NaN  0.030888   

vaccine                     seasonal_vaccine                            \
method                                forest mutual_info matthews_corr   
h1n1_concern                        0.012515    0.023169           NaN   
h1n1_knowledge                      0.016617    0.010745           NaN   
behavioral_antiviral_meds           0.000668    0.000000     -0.000019   
behavioral_avoidance                0.000737    0.000145      0.069538   
behavioral_face_mask                0.000815    0.015302      0.048361   
behavioral_wash_hands               0.001473    0.020270      0.138967   
behavioral_large_gatherings         0.000499    0.000709      0.034773   
behavioral_outside_home             0.000709    0.000893      0.035823   
behavioral_touch_face               0.001849    0.011881      0.118539   
doctor_recc_h1n1                    0.127800    0.023157      0.212517   
doctor_recc_seasonal                0.174299    0.055879      0.338884   
chronic_med_condition               0.009102    0.007763      0.130180   
chi

In [349]:
fig = px.imshow(big_df['seasonal_vaccine'].sort_values('relieff', ascending=False).transpose(),
                color_continuous_scale='balance', color_continuous_midpoint=0,
                aspect='auto', height=450)
fig.show()

In [376]:
fig = px.line(big_df['h1n1_vaccine'].sort_values('relieff'))
fig.show()